In [1]:
# single (first argument) dispatch generic functions

In [2]:
from html import escape


def html_escape(arg) -> str:
    return escape(str(arg))

def html_int(a: int):
    return f"{a}(<i>{hex(a)}</i>"

def html_real(a: float):
    return "{0:.2f}".fromat(round(a, 2))

def html_str(s: str):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l: list):
    items = (f"<li>{html_escape(item)}</li>" for item in l)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d: dict):
    items = (f"<li>{k}={v}</li>" for k, v in d.items())
    return "<ul>\n" + "\n".join(items) + "\n</ul>"


In [3]:
print(html_str("""this is 
a multi line string 
with special chars: 10 < 100
"""))

this is <br/>
a multi line string <br/>
with special chars: 10 &lt; 100<br/>



In [4]:
print(html_int(255))

255(<i>0xff</i>


In [5]:
print(html_escape(3+10j))

(3+10j)


In [6]:
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)

    if isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)

    if isinstance(arg, str):
        return html_str(arg)

    if isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)

    if isinstance(arg, dict):
        return html_dict(arg)

    return html_escape(arg)

        

In [7]:
htmlize("""Python
rocks!""")


'Python<br/>\nrocks!'

In [8]:
print(htmlize([1,2,3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [9]:
print(htmlize(["Hello Bob!", (10, 20, 30), 100]))

<ul>
<li>Hello Bob!</li>
<li>(10, 20, 30)</li>
<li>100</li>
</ul>


In [10]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)

    if isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)

    if isinstance(arg, str):
        return html_str(arg)

    if isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)

    if isinstance(arg, dict):
        return html_dict(arg)

    return html_escape(arg)


def html_escape(arg) -> str:
    return escape(str(arg))

def html_int(a: int):
    return f"{a}(<i>{hex(a)}</i>"

def html_real(a: float):
    return "{0:.2f}".fromat(round(a, 2))

def html_str(s: str):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l: list):
    items = (f"<li>{htmlize(item)}</li>" for item in l)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d: dict):
    items = (f"<li>{html_escape(k)}={htmlize(v)}</li>" for k, v in d.items())
    return "<ul>\n" + "\n".join(items) + "\n</ul>"



In [11]:
print(htmlize(["Hello Bob!", (10, 20, 30), 100, {"a": 20, "b": (1, 2, 3)}]))

<ul>
<li>Hello Bob!</li>
<li><ul>
<li>10(<i>0xa</i></li>
<li>20(<i>0x14</i></li>
<li>30(<i>0x1e</i></li>
</ul></li>
<li>100(<i>0x64</i></li>
<li><ul>
<li>a=20(<i>0x14</i></li>
<li>b=<ul>
<li>1(<i>0x1</i></li>
<li>2(<i>0x2</i></li>
<li>3(<i>0x3</i></li>
</ul></li>
</ul></li>
</ul>


In [12]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        set: html_list,
        tuple: html_list,
        dict: html_dict,
    }

    return registry.get(type(arg), registry[object])(arg)


In [13]:
htmlize(100)

'100(<i>0x64</i>'

In [14]:
print(htmlize({"a": 100, "b<2": "abcd", (1, 2): "200 + 300"}))

<ul>
<li>a=100(<i>0x64</i></li>
<li>b&lt;2=abcd</li>
<li>(1, 2)=200 + 300</li>
</ul>


In [15]:
from functools import wraps


def single_dispatch(fn):
    registry = {
        object: fn,
    }

    @wraps(fn)
    def inner(arg):  # single dispatch (single argument)
        return registry[object](arg)
        
    return inner


In [16]:
@single_dispatch
def htmlize(a):
    return escape(str(a))



In [17]:
htmlize("1 < 100")

'1 &lt; 100'

In [18]:
from functools import wraps


def single_dispatch(fn):
    registry = {
        object: fn,
        int: lambda a: f"{a}(<i>{hex(a)}</i>",
        str: lambda s: escape(s).replace("\n", "<br/>\n"),
    }

    @wraps(fn)
    def inner(arg):  # single dispatch (single argument)
        return registry.get(type(arg), registry[object])(arg)
        
    return inner


In [19]:
@single_dispatch
def htmlize(arg):
    return escape(str(arg))

In [20]:
htmlize("1 < 100")

'1 &lt; 100'

In [21]:
htmlize(200)

'200(<i>0xc8</i>'

In [22]:
htmlize((1, 2, 3, 4))

'(1, 2, 3, 4)'

In [23]:
def single_dispatch(fn):
    registry = {
        object: fn,
    }

    def decorated(arg):  # single dispatch (single argument)
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):  # parametrized decorator -> decorator factory
        def inner(fn):
            registry[type_] = fn
            print(registry)
            return fn
        return inner

    decorated.register = register
    decorated.registry = registry  # not the best idea to allow users to accss it
    return decorated

In [24]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [25]:
htmlize("1 < 100")

'1 &lt; 100'

In [26]:
htmlize.__dict__  # register function from decorator is available!

{'register': <function __main__.single_dispatch.<locals>.register(type_)>,
 'registry': {object: <function __main__.htmlize(a)>}}

In [27]:
@htmlize.register(int)  # adds the function to the htmlize registry!
def html_int(a):
    return f"{a}(<i>{hex(a)}</i>"

{<class 'object'>: <function htmlize at 0x10510f420>, <class 'int'>: <function html_int at 0x10510fd80>}


In [28]:
htmlize(100)

'100(<i>0x64</i>'

In [29]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_list(arg):
    # same as html_list = htmlize.register(list)(html_list)
    items = (f"<li>{htmlize(item)}</li>" for item in arg)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

{<class 'object'>: <function htmlize at 0x10510f420>, <class 'int'>: <function html_int at 0x10510fd80>, <class 'list'>: <function html_list at 0x105148360>}
{<class 'object'>: <function htmlize at 0x10510f420>, <class 'int'>: <function html_int at 0x10510fd80>, <class 'list'>: <function html_list at 0x105148360>, <class 'tuple'>: <function html_list at 0x105148360>}


In [30]:
htmlize([1, 2, 3, 4])

'<ul>\n<li>1(<i>0x1</i></li>\n<li>2(<i>0x2</i></li>\n<li>3(<i>0x3</i></li>\n<li>4(<i>0x4</i></li>\n</ul>'

In [31]:
htmlize((1, 2, 3, 4))

'<ul>\n<li>1(<i>0x1</i></li>\n<li>2(<i>0x2</i></li>\n<li>3(<i>0x3</i></li>\n<li>4(<i>0x4</i></li>\n</ul>'

In [32]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_list(arg)>,
 tuple: <function __main__.html_list(arg)>}

In [33]:
def single_dispatch(fn):
    registry = {
        object: fn,
    }

    def decorated(arg):  # single dispatch (single argument)
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):  # parametrized decorator -> decorator factory
        def inner(fn):
            registry[type_] = fn
            print(registry)
            return fn
        return inner

    def dispatch(type_ = object):
        return registry.get(type_, registry[object])

    
    decorated.register = register
    decorated.dispatch = dispatch
    return decorated

In [34]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [35]:
htmlize.dispatch()

<function __main__.htmlize(a)>

In [36]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_list(arg):
    # same as html_list = htmlize.register(list)(html_list)
    items = (f"<li>{htmlize(item)}</li>" for item in arg)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

{<class 'object'>: <function htmlize at 0x105148cc0>, <class 'list'>: <function html_list at 0x105149620>}
{<class 'object'>: <function htmlize at 0x105148cc0>, <class 'list'>: <function html_list at 0x105149620>, <class 'tuple'>: <function html_list at 0x105149620>}


In [37]:
htmlize.dispatch(list)

<function __main__.html_list(arg)>

In [38]:
htmlize.dispatch(tuple)

<function __main__.html_list(arg)>

In [39]:
from numbers import Integral

@single_dispatch
def htmlize(a):
    return escape(str(a))


@htmlize.register(Integral)
def html_integral_number(a):
    return f"{a}(<i>{hex(a)}</i>"

{<class 'object'>: <function htmlize at 0x105149da0>, <class 'numbers.Integral'>: <function html_integral_number at 0x105148d60>}


In [40]:
htmlize(10)  # doesn't work because type(10) is int, not Integral. Decorator needs to be fixed

'10'

In [41]:
from functools import singledispatch

help(singledispatch)

Help on function singledispatch in module functools:

singledispatch(func)
    Single-dispatch generic function decorator.

    Transforms a function into a generic function, which can have different
    behaviours depending upon the type of its first argument. The decorated
    function acts as the default implementation, and additional
    implementations can be registered using the register() attribute of the
    generic function.



In [42]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [43]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [44]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [45]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f"{a}(<i>{hex(a)}</i>"
    

In [46]:
htmlize(100)

'100(<i>0x64</i>'

In [47]:
htmlize(True)

'True(<i>0x1</i>'

In [48]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.html_integral_number(a)>})

In [49]:
htmlize.dispatch(int)

<function __main__.html_integral_number(a)>

In [50]:
htmlize("1 < 100")

'1 &lt; 100'

In [51]:
from collections.abc import Sequence

@htmlize.register(Sequence)
def html_sequence(arg):
    return " Sequence! <br/>".join(htmlize(item) for item in arg)

In [52]:
htmlize([1, 2, 3, 4])

'1(<i>0x1</i> Sequence! <br/>2(<i>0x2</i> Sequence! <br/>3(<i>0x3</i> Sequence! <br/>4(<i>0x4</i>'

In [53]:
try:
    htmlize("abcd")  # recursion error!! htmlize is called in loop - string is a Sequence!
except RecursionError as e:
    print(e)

maximum recursion depth exceeded


In [54]:
# to fix, we need to register more specific type, str in this case
@htmlize.register  # no explicit type, but function signature has a type and that will be used
def html_str(s: str):
    return f"<br/>{escape(s)}<br/>"

In [55]:
htmlize("abc")

'<br/>abc<br/>'